## Testing Elevation Drive

Craig Lage - 04-Jul-24


In [ ]:
import sys, time, os, asyncio

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import logging

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
#latiss = LATISS(domain)
await asyncio.gather(atcs.start_task)#, latiss.start_task)

In [ ]:
await atcs.enable()

In [ ]:
current_az = 0.0
current_el = 80.0
current_rot = 0.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = -160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = 160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = -160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = 0.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = 80.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

Now we'll analyze the data.

We'll access the EFD instance deployed at the Summit.

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
#t_end = Time(time.time(),format='unix', scale='utc')
t_end = Time("2024-07-04T21:05:00", scale='utc')   # timestamp at end of desired data 
nsec = 0.6*3600 # how many seconds of data to retrieve
torque = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", 'elevationMotorTorque',
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
angle = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 'elevationCalculatedAngle',
                                              t_end - TimeDelta(nsec, format='sec'), t_end)

vel = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", 'elevationMotorVelocity',
                                              t_end - TimeDelta(nsec, format='sec'), t_end)

err = await client.select_time_series("lsst.sal.ATMCS.logevent_errorCode", 'errorCode',
                                              t_end - TimeDelta(nsec, format='sec'), t_end)


In [ ]:
print(len(torque), len(angle), len(vel), len(err))

In [ ]:
%matplotlib inline
ax1 = torque['elevationMotorTorque'].plot(figsize=(16,8), title='Elevation motor torque', legend=True)
#ax1 = vel['elevationMotorVelocity'].plot(figsize=(16,8), title='Elevation motor torque', legend=True)
ax1.axhline(3.0, color="blue", linestyle="--")
ax1.axhline(-3.0, color="blue", linestyle="--")
plt.ylim(-3.5,3.5)
angle['elevationCalculatedAngle'].plot(legend=True, secondary_y=True)
#plt.savefig("/home/cslage/DATA/Elevation_Torque_04Jul24.png")


In [ ]:
%matplotlib inline
ax1 = vel['elevationMotorVelocity'].plot(figsize=(16,8), title='Elevation motor velocity', legend=True)
for t in err.index:
    for t2 in angle.index:
        if t2.to_julian_date() > t.to_julian_date():
            break
    index = angle.index.searchsorted(t2)
    fail = angle.iloc[index]['elevationCalculatedAngle']
    ax1.axvline(t, color="red", linestyle="--")
    ax1.text(t, -3.0, f"     {fail:.2f} degrees")
plt.ylim(-5, 5)
angle['elevationCalculatedAngle'].plot(legend=True, secondary_y=True)
#plt.savefig("/home/cslage/DATA/Elevation_Velocity_04Jul24.png")


In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,8))
ax1 = vel['elevationMotorVelocity'].plot(figsize=(16,8), title='Elevation motor velocity', legend=True)
for t in err.index:
    for t2 in angle.index:
        if t2.to_julian_date() > t.to_julian_date():
            break
    index = angle.index.searchsorted(t2)
    fail = angle.iloc[index]['elevationCalculatedAngle']
    ax1.axvline(t, color="red", linestyle="--")
    ax1.text(t, -3.0, f"     {fail:.2f} degrees")
plt.ylim(-5, 5)
angle['elevationCalculatedAngle'].plot(legend=True, secondary_y=True)
plt.xlim(Time("2024-07-04T20:34:03", scale='utc').isot, Time("2024-07-04T20:34:40", scale='utc').isot)
#plt.savefig("/home/cslage/DATA/Elevation_Velocity_04Jul24.png")


In [ ]:
for t in err.index:
    for t2 in angle.index:
        if t2.to_julian_date() > t.to_julian_date():
            print(t2)
            break
    index = angle.index.searchsorted(t2)
    print(angle.iloc[index]['elevationCalculatedAngle'])
    

In [ ]:
t2.to_julian_date()

In [ ]:
torqueList = torque.values.tolist()
angleList = angle.values.tolist()
plt.figure()
plt.plot(np.array(angleList)[:,0],np.array(torqueList)[:,0])
plt.plot([0,90],[3.0,3.0], color='red', ls='--')
plt.plot([0,90],[-3.0,-3.0], color='red', ls='--')
#plt.arrow(20, 2.5, 10,0, width=0.1,head_length = 5.0, color='green')
#plt.arrow(60, -2.5, -10,0, width=0.1,head_length = 5.0, color='green')
plt.xlabel("Rotator angle(degrees)")
plt.ylabel("Torque (amps)")
#plt.savefig("/home/craiglagegit/DATA/Torque_vs_Angle_12Jul22.pdf")

In [ ]:
velList = vel.values.tolist()
angleList = angle.values.tolist()
plt.figure()
plt.plot(np.array(angleList)[:,0],np.array(velList)[:,0])
#plt.plot([0,90],[3.0,3.0], color='red', ls='--')
#plt.plot([0,90],[-3.0,-3.0], color='red', ls='--')
#plt.arrow(20, 2.5, 10,0, width=0.1,head_length = 5.0, color='green')
#plt.arrow(60, -2.5, -10,0, width=0.1,head_length = 5.0, color='green')
plt.xlabel("Rotator angle(degrees)")
plt.ylabel("Velocity ")
#plt.savefig("/home/craiglagegit/DATA/Torque_vs_Angle_12Jul22.pdf")